# Assignment 5: 


##Task-1
### Fine-tuning resnet18 on X-Ray Chest data
In this tutorial we will learn how to fine-tune a pre-trained network on a new dataset.
We will perform the following steps:
1. Load and normalizing the X-Ray dataset
2. Load pre-trained ResNet18 
3. Remove top layers (fully connected layers)
4. Freeze the network
4. Add new layers (classifier)
5. Train the network
6. Plot Loss & Accuracies with Epochs
7. Confusion Matrix
8. F1 Score
9. Test Final Accuracy on test data

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from google.colab import drive


### Load Dataset usign torchvision image loader

In [0]:
drive.mount('/content/drive')
# drive.mount("./gdrive")
# data_dir = '/content/drive/My Drive/MSDS/Semester 4/DL/Homeworks/Homework 5/Assignment 5 Dataset'

!unzip "/content/drive/My Drive/Assignment 5 Dataset.zip"

In [0]:
data_dir = "Assignment 5 Dataset"
#Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

#pass transform here-in
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/validation', transform=test_transforms)

#data loaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=1500, shuffle=True)

print("Classes: ")
class_names = train_data.classes
print(class_names)

In [0]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    plt.axis('off')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

def show_databatch(inputs, classes):
    out = torchvision.utils.make_grid(inputs)
    imshow(out, title=[class_names[x] for x in classes])

# Get a batch of training data
inputs, classes = next(iter(trainloader))
show_databatch(inputs, classes)

## ResNet-18

### Load pre-trained ResNet18

In [0]:
# Load the pretrained model from pytorch
resnet18 = models.resnet18(pretrained=True)
print(resnet18)
# print('Output Layer of resnet18 : ', resnet18.classifier[6].out_features) # 1000 

In [0]:
print(resnet18.fc)

### Removing Last Layer

In [0]:
num_features = resnet18.fc.in_features # 0 means input we are receiving from VGG Conv features
# putting array value 0 instead of -1 (which is used to neglect last layer) to add new layers
features = list(resnet18.fc.children())[:0] # Remove all FC layer
print(num_features)

### Freezing the layers

In [0]:
# Freeze training for all layers
for param in resnet18.parameters():
    param.requires_grad = False

### Adding New Layer

In [0]:
roll_number_neurons = 22*10+100
features.extend([
                 nn.Linear(num_features, roll_number_neurons)
                 ,nn.ReLU(inplace=True)
                 ,nn.Linear(roll_number_neurons, len(class_names))
                 ])

In [0]:
resnet18.fc = nn.Sequential(*features)
print(resnet18)

### Loss fucntion and optimizer

In [0]:
Epochs = 5
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=0.0001, momentum=0.9)

### Training

In [0]:
from tqdm import tqdm
since = time.time()
#if you have gpu then you need to convert the network and data to cuda
#the easiest way is to first check for device and then convert network and data to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
resnet18.to(device)

# resnet18.train()
loss_plot = []
loss_plot_val = []
accuracy_plot = []
accuracy_plot_val=[]

for epoch in range(Epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    running_corrects = 0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # if i==4:
        #   break
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation 
        # because PyTorch accumulates the gradients on subsequent backward passes. 
        # This is convenient while training RNNs. 
        # So, the default action is to accumulate the gradients on every loss.backward() call

        # forward + backward + optimize
        outputs = resnet18(inputs)               #----> forward pass
        _, preds = torch.max(outputs, 1) #added line for accuracy
        loss = criterion(outputs, labels)   #----> compute loss
        
        loss.backward()                     #----> backward pass
        optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels.data)
        
        train_data_len = len(trainloader.dataset)
        epoch_loss = running_loss / train_data_len
        epoch_acc = running_corrects.double() / train_data_len
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tTraining Loss: {:.4f}  Training Acc: {:.4f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                epoch_loss, epoch_acc))
        # pbar.set_description(
        #     'Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc)
        #     )
    
    

    loss_plot.append(epoch_loss)
    
    accuracy_plot.append(epoch_acc)
    correct = 0
    total = 0
    with torch.no_grad():
        running_loss_val = 0.0
        running_corrects_val=0.0
        for data in tqdm(testloader):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = resnet18(images)               #----> forward pass
            _, preds = torch.max(outputs, 1) #added line for accuracy
            loss_val = criterion(outputs, labels)
            
            running_loss_val += loss_val.item()
            running_corrects_val += torch.sum(preds == labels.data)


        test_data_len = len(testloader.dataset)
        epoch_loss = running_loss_val / test_data_len
        epoch_acc = running_corrects_val.double() / test_data_len

        loss_plot_val.append(epoch_loss)
        accuracy_plot_val.append(epoch_acc)
    
    time_elapsed = time.time() - since
        # print(loss)
    torch.save(resnet18.state_dict(), 'resnet18_FC_Only.pth')

print('\n\nTraining complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

### Plot Loss & Accuracies

In [0]:

plt.title("Training Accuracy")
plt.xlabel("Training Epochs")
plt.ylabel("Training Accuracy")
plt.xticks(np.arange(1, Epochs+1, 1.0))
plt.plot(range(1,Epochs+1),accuracy_plot,label="Training Accuracy")
plt.plot(range(1,Epochs+1),accuracy_plot_val,label="Validation Accuracy")
plt.legend()
plt.show()

plt.title("Training Loss")
plt.xlabel("Training Epochs")
plt.ylabel("Training Loss")
plt.xticks(np.arange(1, Epochs+1, 1.0))
plt.plot(range(1,Epochs+1),loss_plot,label="Training Loss")
plt.plot(range(1,Epochs+1),loss_plot_val,label="Validation Loss")
plt.legend()
plt.show()


### Confusion Matrix & Two Worse and Best Images

In [0]:
from sklearn.metrics import confusion_matrix, f1_score
number_of_classes = 2
confusion_matrix = torch.zeros(number_of_classes, number_of_classes)
with torch.no_grad():
    for i, (inputs, label) in enumerate(testloader):
        inputs = inputs.to(device)
        label = label.to(device)
        outputs = resnet18(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(label.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

### F1 Score

In [0]:
for i,(inputs,label) in enumerate(testloader):
        inputs = inputs.to(device)
        label = label.to(device)
        # compute output
        output = resnet18(inputs)
        loss = criterion(output,label)
        _, preds = torch.max(output, 1)
        # losses.update(loss.item(),input.size(0))
        f1_batch = f1_score(label.cpu(),preds.cpu() > 0.15,average='macro')
print('F1 Score: ',f1_batch)

### Final Accuracy

In [0]:
dataiter = iter(trainloader) # converted it to train because test batch is of 1500 (converted to increase speed)
images, labels = dataiter.next()
show_databatch(images, labels)

In [0]:
images, labels = images.to(device), labels.to(device) #-->convert test image to cuda (if available)
outputs = resnet18(images)                               #--> forward pass
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % class_names[predicted[j]]
                              for j in range(len(images))))
print('Ground Truth: ', ' '.join('%5s' % class_names[labels[j]]
                              for j in range(len(images))))

In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = resnet18(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Final Accuracy of the network on the 1500 test images: %d %%' % (
    100 * correct / total))

### Best and Worse Performing Images

In [0]:
correct = 0
total = 0
best_performing_image1 = 0
best_performing_image2 = 0
best_performing_image1_label = ""
best_performing_image2_label = ""

worse_performing_image1 = 0
worse_performing_image2 = 0
worse_performing_image1_label = ""
worse_performing_image2_label = ""

maximum = 0
minimum = -9999

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = resnet18(images)
        probabilities, predicted = torch.max(outputs.data,1)

        T_Pred_Mask= probabilities[predicted==labels]
        F_Pred_Mask= probabilities[predicted!=labels]

        if any(T_Pred_Mask) == True:
          temp_max = torch.argmax(probabilities[predicted==labels])
          if maximum < temp_max:
            maximum = temp_max
            best_performing_image2 = best_performing_image1
            best_performing_image2_label = best_performing_image1_label

            best_performing_image1_label = labels[maximum]
            best_performing_image1 = images[maximum]
        
        if any(F_Pred_Mask) == True:
          temp_min = torch.argmin(probabilities[predicted!=labels])
          if minimum > temp_min  :
            minimum = temp_min
            worse_performing_image2 = worse_performing_image1
            worse_performing_image2_label = worse_performing_image1_label

            worse_performing_image1_label = labels[minimum]
            worse_performing_image1 = images[minimum]

        # total += labels.size(0)
        # correct += (predicted == labels).sum().item()
print('True Positive or True Negative with High Probability')
imshow(best_performing_image1.cpu(),best_performing_image1_label.cpu())
imshow(best_performing_image2.cpu(),best_performing_image2_label.cpu())

print('False Positive or False Negative with High Probability')
imshow(worse_performing_image1.cpu(),worse_performing_image1_label.cpu())
imshow(worse_performing_image2.cpu(),worse_performing_image2_label.cpu())

## VGG-16
### Fine-tuning vgg16 on X-Ray Chest data
In this tutorial we will learn how to fine-tune a pre-trained network on a new dataset.
We will perform the following steps:
1. Load and normalizing the X-Ray dataset
2. Load pre-trained Vgg16 
3. Remove top layers (fully connected layers)
4. Freeze the network
4. Add new layers (classifier)
5. Train the network
6. Plot Loss & Accuracies with Epochs
7. Confusion Matrix
8. F1 Score
9. Test Final Accuracy on test data

### Load pre-trained VGG-16

In [0]:
# Load the pretrained model from pytorch
vgg16 = models.vgg16(pretrained=True)
print(vgg16)
print('Output Layer of VGG16 : ', vgg16.classifier[6].out_features) # 1000 

In [0]:
print(vgg16.classifier)

In [0]:
num_features = vgg16.classifier[0].in_features # 0 means input we are receiving from VGG Conv features
# putting array value 0 instead of -1 (which is used to neglect last layer) to add new layers
features = list(vgg16.classifier.children())[:0] # Remove all FC layer
print(num_features)

### Freezing the layers

In [0]:
# Freeze training for all layers
for param in vgg16.features.parameters():
    param.requires_grad = False

### Adding New Layer

In [0]:
roll_number_neurons = 22*10+100
features.extend([
                 nn.Linear(num_features, roll_number_neurons)
                 ,nn.ReLU(inplace=True)
                 ,nn.Linear(roll_number_neurons, len(class_names))
                 ])

In [0]:
vgg16.classifier = nn.Sequential(*features)
print(vgg16)

### Loss fucntion and optimizer

In [0]:
Epochs = 3
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)

### Training

In [0]:
from tqdm import tqdm
since = time.time()
#if you have gpu then you need to convert the network and data to cuda
#the easiest way is to first check for device and then convert network and data to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
vgg16.to(device)

# vgg16.train()
loss_plot = []
loss_plot_val = []
accuracy_plot = []
accuracy_plot_val=[]

for epoch in range(Epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    running_corrects = 0
    epoch_loss=0.0
    epoch_acc=0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # if i==20: #my code was failing due to CUDA memory, so doing this to train on subset
        #   break
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation 
        # because PyTorch accumulates the gradients on subsequent backward passes. 
        # This is convenient while training RNNs. 
        # So, the default action is to accumulate the gradients on every loss.backward() call

        # forward + backward + optimize
        outputs = vgg16(inputs)               #----> forward pass
        _, preds = torch.max(outputs, 1) #added line for accuracy
        loss = criterion(outputs, labels)   #----> compute loss
        
        loss.backward()                     #----> backward pass
        optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels.data)
        train_data_len = len(trainloader.dataset)
        epoch_loss = running_loss / train_data_len
        epoch_acc = running_corrects.double() / train_data_len
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tTraining Loss: {:.4f}  Training Acc: {:.4f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                epoch_loss, epoch_acc))
        # pbar.set_description(
        #     'Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc)
        #     )
    
    train_data_len = len(trainloader.dataset)
    epoch_loss = running_loss / train_data_len
    epoch_acc = running_corrects.double() / train_data_len

    loss_plot.append(epoch_loss)
    
    accuracy_plot.append(epoch_acc)
    correct = 0
    total = 0
    with torch.no_grad():
        running_loss_val = 0.0
        running_corrects_val=0.0
        for data in tqdm(testloader):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)               #----> forward pass
            _, preds = torch.max(outputs, 1) #added line for accuracy
            loss_val = criterion(outputs, labels)
            
            running_loss_val += loss_val.item()
            running_corrects_val += torch.sum(preds == labels.data)


        test_data_len = len(testloader.dataset)
        epoch_loss = running_loss_val / test_data_len
        epoch_acc = running_corrects_val.double() / test_data_len

        loss_plot_val.append(epoch_loss)
        accuracy_plot_val.append(epoch_acc)
    
    time_elapsed = time.time() - since
        # print(loss)
    torch.save(vgg16.state_dict(), 'vgg16_FC_Only.pth')

print('\n\nTraining complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

### Plot Loss & Accuracies

In [0]:

plt.title("Training Accuracy")
plt.xlabel("Training Epochs")
plt.ylabel("Training Accuracy")
plt.xticks(np.arange(1, Epochs+1, 1.0))
plt.plot(range(1,Epochs+1),accuracy_plot,label="Training Accuracy")
plt.plot(range(1,Epochs+1),accuracy_plot_val,label="Validation Accuracy")
plt.legend()
plt.show()

plt.title("Training Loss")
plt.xlabel("Training Epochs")
plt.ylabel("Training Loss")
plt.xticks(np.arange(1, Epochs+1, 1.0))
plt.plot(range(1,Epochs+1),loss_plot,label="Training Loss")
plt.plot(range(1,Epochs+1),loss_plot_val,label="Validation Loss")
plt.legend()
plt.show()


### Confusion Matrix & Two Worse and Best Images

In [0]:
from sklearn.metrics import confusion_matrix, f1_score
number_of_classes = 2
confusion_matrix = torch.zeros(number_of_classes, number_of_classes)
with torch.no_grad():
    for i, (inputs, label) in enumerate(testloader):
        inputs = inputs.to(device)
        label = label.to(device)
        outputs = vgg16(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(label.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

### F1 Score

In [0]:
for i,(inputs,label) in enumerate(testloader):
        inputs = inputs.to(device)
        label = label.to(device)
        # compute output
        output = vgg16(inputs)
        loss = criterion(output,label)
        _, preds = torch.max(output, 1)
        # losses.update(loss.item(),input.size(0))
        f1_batch = f1_score(label.cpu(),preds.cpu() > 0.15,average='macro')
print('F1 Score: ',f1_batch)

### Final Accuracy

In [0]:
dataiter = iter(trainloader) # converted it to train because test batch is of 1500 (converted to increase speed)
images, labels = dataiter.next()
show_databatch(images, labels)

In [0]:
images, labels = images.to(device), labels.to(device) #-->convert test image to cuda (if available)
outputs = vgg16(images)                               #--> forward pass
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % class_names[predicted[j]]
                              for j in range(len(images))))
print('Ground Truth: ', ' '.join('%5s' % class_names[labels[j]]
                              for j in range(len(images))))

In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = vgg16(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Final Accuracy of the network on the 1500 test images: %d %%' % (
    100 * correct / total))

### Best and Worse Performing Images

In [0]:
correct = 0
total = 0
best_performing_image1 = 0
best_performing_image2 = 0
best_performing_image1_label = ""
best_performing_image2_label = ""

worse_performing_image1 = 0
worse_performing_image2 = 0
worse_performing_image1_label = ""
worse_performing_image2_label = ""

maximum = 0
minimum = -9999

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = resnet18(images)
        probabilities, predicted = torch.max(outputs.data,1)

        T_Pred_Mask= probabilities[predicted==labels]
        F_Pred_Mask= probabilities[predicted!=labels]

        if any(T_Pred_Mask) == True:
          temp_max = torch.argmax(probabilities[predicted==labels])
          if maximum < temp_max:
            maximum = temp_max
            best_performing_image2 = best_performing_image1
            best_performing_image2_label = best_performing_image1_label

            best_performing_image1_label = labels[maximum]
            best_performing_image1 = images[maximum]
        
        if any(F_Pred_Mask) == True:
          temp_min = torch.argmin(probabilities[predicted!=labels])
          if minimum > temp_min  :
            minimum = temp_min
            worse_performing_image2 = worse_performing_image1
            worse_performing_image2_label = worse_performing_image1_label

            worse_performing_image1_label = labels[minimum]
            worse_performing_image1 = images[minimum]

        # total += labels.size(0)
        # correct += (predicted == labels).sum().item()
print('True Positive or True Negative with High Probability')
imshow(best_performing_image1.cpu(),best_performing_image1_label.cpu())
imshow(best_performing_image2.cpu(),best_performing_image2_label.cpu())

print('False Positive or False Negative with High Probability')
imshow(worse_performing_image1.cpu(),worse_performing_image1_label.cpu())
imshow(worse_performing_image2.cpu(),worse_performing_image2_label.cpu())

# Task 2

## ResNet-18

### Load pre-trained ResNet18

In [0]:
# Load the pretrained model from pytorch
resnet18 = models.resnet18(pretrained=True)
print(resnet18)
# print('Output Layer of resnet18 : ', resnet18.classifier[6].out_features) # 1000 

In [0]:
print(resnet18.fc)

### Removing Last Layer

In [0]:
num_features = resnet18.fc.in_features # 0 means input we are receiving from VGG Conv features
# putting array value 0 instead of -1 (which is used to neglect last layer) to add new layers
features = list(resnet18.fc.children())[:0] # Remove all FC layer
print(num_features)

### Freezing the layers

In [0]:
# Freeze training for all layers
for param in resnet18.parameters():
    param.requires_grad = False

for i in range(0,31): # Unfreezing all CNN Layer
  resnet18.features[i].requires_grad = True

### Adding New Layer

In [0]:
roll_number_neurons = 22*10+100
features.extend([
                 nn.Linear(num_features, roll_number_neurons)
                 ,nn.ReLU(inplace=True)
                 ,nn.Linear(roll_number_neurons, len(class_names))
                 ])

In [0]:
resnet18.fc = nn.Sequential(*features)
print(resnet18)

### Loss fucntion and optimizer

In [0]:
Epochs = 5
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=0.0001, momentum=0.9)

### Training

In [0]:
from tqdm import tqdm
since = time.time()
#if you have gpu then you need to convert the network and data to cuda
#the easiest way is to first check for device and then convert network and data to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
resnet18.to(device)

# resnet18.train()
loss_plot = []
loss_plot_val = []
accuracy_plot = []
accuracy_plot_val=[]

for epoch in range(Epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    running_corrects = 0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # if i==4:
        #   break
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation 
        # because PyTorch accumulates the gradients on subsequent backward passes. 
        # This is convenient while training RNNs. 
        # So, the default action is to accumulate the gradients on every loss.backward() call

        # forward + backward + optimize
        outputs = resnet18(inputs)               #----> forward pass
        _, preds = torch.max(outputs, 1) #added line for accuracy
        loss = criterion(outputs, labels)   #----> compute loss
        
        loss.backward()                     #----> backward pass
        optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels.data)
        
        train_data_len = len(trainloader.dataset)
        epoch_loss = running_loss / train_data_len
        epoch_acc = running_corrects.double() / train_data_len

        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tTraining Loss: {:.4f}  Training Acc: {:.4f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                epoch_loss, epoch_acc))
        # pbar.set_description(
        #     'Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc)
        #     )

    loss_plot.append(epoch_loss)
    
    accuracy_plot.append(epoch_acc)
    correct = 0
    total = 0
    with torch.no_grad():
        running_loss_val = 0.0
        running_corrects_val=0.0
        for data in tqdm(testloader):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = resnet18(images)               #----> forward pass
            _, preds = torch.max(outputs, 1) #added line for accuracy
            loss_val = criterion(outputs, labels)
            
            running_loss_val += loss_val.item()
            running_corrects_val += torch.sum(preds == labels.data)


        test_data_len = len(testloader.dataset)
        epoch_loss = running_loss_val / test_data_len
        epoch_acc = running_corrects_val.double() / test_data_len

        loss_plot_val.append(epoch_loss)
        accuracy_plot_val.append(epoch_acc)
    
    time_elapsed = time.time() - since
        # print(loss)
    torch.save(resnet18.state_dict(), 'resnet18_FC_Only.pth')

print('\n\nTraining complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

### Plot Loss & Accuracies

In [0]:

plt.title("Training Accuracy")
plt.xlabel("Training Epochs")
plt.ylabel("Training Accuracy")
plt.xticks(np.arange(1, Epochs+1, 1.0))
plt.plot(range(1,Epochs+1),accuracy_plot,label="Training Accuracy")
plt.plot(range(1,Epochs+1),accuracy_plot_val,label="Validation Accuracy")
plt.legend()
plt.show()

plt.title("Training Loss")
plt.xlabel("Training Epochs")
plt.ylabel("Training Loss")
plt.xticks(np.arange(1, Epochs+1, 1.0))
plt.plot(range(1,Epochs+1),loss_plot,label="Training Loss")
plt.plot(range(1,Epochs+1),loss_plot_val,label="Validation Loss")
plt.legend()
plt.show()


### Confusion Matrix & Two Worse and Best Images

In [0]:
from sklearn.metrics import confusion_matrix, f1_score
number_of_classes = 2
confusion_matrix = torch.zeros(number_of_classes, number_of_classes)
with torch.no_grad():
    for i, (inputs, label) in enumerate(testloader):
        inputs = inputs.to(device)
        label = label.to(device)
        outputs = resnet18(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(label.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

### F1 Score

In [0]:
for i,(inputs,label) in enumerate(testloader):
        inputs = inputs.to(device)
        label = label.to(device)
        # compute output
        output = resnet18(inputs)
        loss = criterion(output,label)
        _, preds = torch.max(output, 1)
        # losses.update(loss.item(),input.size(0))
        f1_batch = f1_score(label.cpu(),preds.cpu() > 0.15,average='macro')
print('F1 Score: ',f1_batch)

### Final Accuracy

In [0]:
dataiter = iter(trainloader) # converted it to train because test batch is of 1500 (converted to increase speed)
images, labels = dataiter.next()
show_databatch(images, labels)

In [0]:
images, labels = images.to(device), labels.to(device) #-->convert test image to cuda (if available)
outputs = resnet18(images)                               #--> forward pass
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % class_names[predicted[j]]
                              for j in range(len(images))))
print('Ground Truth: ', ' '.join('%5s' % class_names[labels[j]]
                              for j in range(len(images))))

In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = resnet18(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Final Accuracy of the network on the 1500 test images: %d %%' % (
    100 * correct / total))

### Best and Worse Performing Images

In [0]:
correct = 0
total = 0
best_performing_image1 = 0
best_performing_image2 = 0
best_performing_image1_label = ""
best_performing_image2_label = ""

worse_performing_image1 = 0
worse_performing_image2 = 0
worse_performing_image1_label = ""
worse_performing_image2_label = ""

maximum = 0
minimum = -9999

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = resnet18(images)
        probabilities, predicted = torch.max(outputs.data,1)

        T_Pred_Mask= probabilities[predicted==labels]
        F_Pred_Mask= probabilities[predicted!=labels]

        if any(T_Pred_Mask) == True:
          temp_max = torch.argmax(probabilities[predicted==labels])
          if maximum < temp_max:
            maximum = temp_max
            best_performing_image2 = best_performing_image1
            best_performing_image2_label = best_performing_image1_label

            best_performing_image1_label = labels[maximum]
            best_performing_image1 = images[maximum]
        
        if any(F_Pred_Mask) == True:
          temp_min = torch.argmin(probabilities[predicted!=labels])
          if minimum > temp_min  :
            minimum = temp_min
            worse_performing_image2 = worse_performing_image1
            worse_performing_image2_label = worse_performing_image1_label

            worse_performing_image1_label = labels[minimum]
            worse_performing_image1 = images[minimum]

        # total += labels.size(0)
        # correct += (predicted == labels).sum().item()
print('True Positive or True Negative with High Probability')
imshow(best_performing_image1.cpu(),best_performing_image1_label.cpu())
imshow(best_performing_image2.cpu(),best_performing_image2_label.cpu())

print('False Positive or False Negative with High Probability')
imshow(worse_performing_image1.cpu(),worse_performing_image1_label.cpu())
imshow(worse_performing_image2.cpu(),worse_performing_image2_label.cpu())

## VGG-16
### Fine-tuning vgg16 on X-Ray Chest data
In this tutorial we will learn how to fine-tune a pre-trained network on a new dataset.
We will perform the following steps:
1. Load and normalizing the X-Ray dataset
2. Load pre-trained Vgg16 
3. Remove top layers (fully connected layers)
4. Freeze the network
4. Add new layers (classifier)
5. Train the network
6. Plot Loss & Accuracies with Epochs
7. Confusion Matrix
8. F1 Score
9. Test Final Accuracy on test data

### Load pre-trained VGG-16

In [0]:
# Load the pretrained model from pytorch
vgg16 = models.vgg16(pretrained=True)
print(vgg16)
print('Output Layer of VGG16 : ', vgg16.classifier[6].out_features) # 1000 

In [0]:
print(vgg16.classifier)

In [0]:
num_features = vgg16.classifier[0].in_features # 0 means input we are receiving from VGG Conv features
# putting array value 0 instead of -1 (which is used to neglect last layer) to add new layers
features = list(vgg16.classifier.children())[:0] # Remove all FC layer
print(num_features)

### Freezing the layers

In [0]:
# Freeze training for all layers
for param in vgg16.features.parameters():
    param.requires_grad = False

for i in range(0,31): # Unfreezing all CNN Layer
  vgg16.features[i].requires_grad = True

### Adding New Layer

In [0]:
roll_number_neurons = 22*10+100
features.extend([
                 nn.Linear(num_features, roll_number_neurons)
                 ,nn.ReLU(inplace=True)
                 ,nn.Linear(roll_number_neurons, len(class_names))
                 ])

In [0]:
vgg16.classifier = nn.Sequential(*features)
print(vgg16)

### Loss fucntion and optimizer

In [0]:
Epochs = 3
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)

### Training

In [0]:
from tqdm import tqdm
since = time.time()
#if you have gpu then you need to convert the network and data to cuda
#the easiest way is to first check for device and then convert network and data to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
vgg16.to(device)

# vgg16.train()
loss_plot = []
loss_plot_val = []
accuracy_plot = []
accuracy_plot_val=[]

for epoch in range(Epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    running_corrects = 0
    epoch_loss=0.0
    epoch_acc=0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # if i==20: #my code was failing due to CUDA memory, so doing this to train on subset
        #   break
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation 
        # because PyTorch accumulates the gradients on subsequent backward passes. 
        # This is convenient while training RNNs. 
        # So, the default action is to accumulate the gradients on every loss.backward() call

        # forward + backward + optimize
        outputs = vgg16(inputs)               #----> forward pass
        _, preds = torch.max(outputs, 1) #added line for accuracy
        loss = criterion(outputs, labels)   #----> compute loss
        
        loss.backward()                     #----> backward pass
        optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels.data)

        train_data_len = len(trainloader.dataset)
        epoch_loss = running_loss / train_data_len
        epoch_acc = running_corrects.double() / train_data_len

        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tTraining Loss: {:.4f}  Training Acc: {:.4f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                epoch_loss, epoch_acc))
        # pbar.set_description(
        #     'Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc)
        #     )
    
    

    loss_plot.append(epoch_loss)
    
    accuracy_plot.append(epoch_acc)
    correct = 0
    total = 0
    with torch.no_grad():
        running_loss_val = 0.0
        running_corrects_val=0.0
        for data in tqdm(testloader):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)               #----> forward pass
            _, preds = torch.max(outputs, 1) #added line for accuracy
            loss_val = criterion(outputs, labels)
            
            running_loss_val += loss_val.item()
            running_corrects_val += torch.sum(preds == labels.data)


        test_data_len = len(testloader.dataset)
        epoch_loss = running_loss_val / test_data_len
        epoch_acc = running_corrects_val.double() / test_data_len

        loss_plot_val.append(epoch_loss)
        accuracy_plot_val.append(epoch_acc)
    
    time_elapsed = time.time() - since
        # print(loss)
    torch.save(vgg16.state_dict(), 'vgg16_FC_Only.pth')

print('\n\nTraining complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

### Plot Loss & Accuracies

In [0]:

plt.title("Training Accuracy")
plt.xlabel("Training Epochs")
plt.ylabel("Training Accuracy")
plt.xticks(np.arange(1, Epochs+1, 1.0))
plt.plot(range(1,Epochs+1),accuracy_plot,label="Training Accuracy")
plt.plot(range(1,Epochs+1),accuracy_plot_val,label="Validation Accuracy")
plt.legend()
plt.show()

plt.title("Training Loss")
plt.xlabel("Training Epochs")
plt.ylabel("Training Loss")
plt.xticks(np.arange(1, Epochs+1, 1.0))
plt.plot(range(1,Epochs+1),loss_plot,label="Training Loss")
plt.plot(range(1,Epochs+1),loss_plot_val,label="Validation Loss")
plt.legend()
plt.show()


### Confusion Matrix & Two Worse and Best Images

In [0]:
from sklearn.metrics import confusion_matrix, f1_score
number_of_classes = 2
confusion_matrix = torch.zeros(number_of_classes, number_of_classes)
with torch.no_grad():
    for i, (inputs, label) in enumerate(testloader):
        inputs = inputs.to(device)
        label = label.to(device)
        outputs = vgg16(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(label.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

### F1 Score

In [0]:
for i,(inputs,label) in enumerate(testloader):
        inputs = inputs.to(device)
        label = label.to(device)
        # compute output
        output = vgg16(inputs)
        loss = criterion(output,label)
        _, preds = torch.max(output, 1)
        # losses.update(loss.item(),input.size(0))
        f1_batch = f1_score(label.cpu(),preds.cpu() > 0.15,average='macro')
print('F1 Score: ',f1_batch)

### Final Accuracy

In [0]:
dataiter = iter(trainloader) # converted it to train because test batch is of 1500 (converted to increase speed)
images, labels = dataiter.next()
show_databatch(images, labels)

In [0]:
images, labels = images.to(device), labels.to(device) #-->convert test image to cuda (if available)
outputs = vgg16(images)                               #--> forward pass
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % class_names[predicted[j]]
                              for j in range(len(images))))
print('Ground Truth: ', ' '.join('%5s' % class_names[labels[j]]
                              for j in range(len(images))))

In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = vgg16(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Final Accuracy of the network on the 1500 test images: %d %%' % (
    100 * correct / total))

### Best and Worse Performing Images

In [0]:
correct = 0
total = 0
best_performing_image1 = 0
best_performing_image2 = 0
best_performing_image1_label = ""
best_performing_image2_label = ""

worse_performing_image1 = 0
worse_performing_image2 = 0
worse_performing_image1_label = ""
worse_performing_image2_label = ""

maximum = 0
minimum = -9999

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = resnet18(images)
        probabilities, predicted = torch.max(outputs.data,1)

        T_Pred_Mask= probabilities[predicted==labels]
        F_Pred_Mask= probabilities[predicted!=labels]

        if any(T_Pred_Mask) == True:
          temp_max = torch.argmax(probabilities[predicted==labels])
          if maximum < temp_max:
            maximum = temp_max
            best_performing_image2 = best_performing_image1
            best_performing_image2_label = best_performing_image1_label

            best_performing_image1_label = labels[maximum]
            best_performing_image1 = images[maximum]
        
        if any(F_Pred_Mask) == True:
          temp_min = torch.argmin(probabilities[predicted!=labels])
          if minimum > temp_min  :
            minimum = temp_min
            worse_performing_image2 = worse_performing_image1
            worse_performing_image2_label = worse_performing_image1_label

            worse_performing_image1_label = labels[minimum]
            worse_performing_image1 = images[minimum]

        # total += labels.size(0)
        # correct += (predicted == labels).sum().item()
print('True Positive or True Negative with High Probability')
imshow(best_performing_image1.cpu(),best_performing_image1_label.cpu())
imshow(best_performing_image2.cpu(),best_performing_image2_label.cpu())

print('False Positive or False Negative with High Probability')
imshow(worse_performing_image1.cpu(),worse_performing_image1_label.cpu())
imshow(worse_performing_image2.cpu(),worse_performing_image2_label.cpu())